In [251]:
import pandas as pd
import statsmodels.api as sm

from sklearn.metrics import mean_squared_error

In [211]:
%run '/Users/louisebonhomme/Documents/GitHub/BDC/Helpers/preprocessing.py'

In [214]:
df = pd.read_csv('/Users/louisebonhomme/Documents/GitHub/BDC/Data/df_Paris_Processed.csv')

## Cleaning iris variables

In [215]:
clean_iris_codes(df)

In [216]:
missing_df = df.isnull().sum(axis=0).reset_index()
missing_df.columns = ['variable', 'missing values']
missing_df['filling factor (%)']=(df.shape[0]-missing_df['missing values'])/df.shape[0]*100
missing_df['type'] = df.dtypes.to_numpy()
missing_df

,variable,missing values,filling factor (%),type
0,id_mutation,0,100.0,object
1,date_mutation,0,100.0,object
2,code_type_local,0,100.0,float64
3,code_commune,0,100.0,int64
4,surface_terrain,0,100.0,float64
5,surface_reelle_bati,0,100.0,float64
6,nombre_pieces_principales,0,100.0,float64
7,nature_culture,0,100.0,object
8,valeur_fonciere,0,100.0,float64
9,latitude,0,100.0,float64


In [217]:
path = "/Users/louisebonhomme/Documents/GitHub/BDC/Data/reference_IRIS_geo2022.xlsx"
dep = '75'
nb_iris(df, path = path, dep_code = dep)

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/Users/louisebonhomme/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3369, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/yz/j7zl8ck90j528mms2yj13vhc0000gn/T/ipykernel_41950/2938108793.py", line 3, in <cell line: 3>
    nb_iris(df, path = path, dep_code = dep)
  File "/Users/louisebonhomme/Documents/GitHub/BDC/Helpers/preprocessing.py", line 59, in nb_iris
    IRIS = pd.read_excel(path, header = 5)[['CODE_IRIS', 'DEP']]
  File "/Users/louisebonhomme/opt/anaconda3/lib/python3.9/site-packages/pandas/util/_decorators.py", line 311, in wrapper
    return func(*args, **kwargs)
  File "/Users/louisebonhomme/opt/anaconda3/lib/python3.9/site-packages/pandas/io/excel/_base.py", line 457, in read_excel
    io = ExcelFile(io, storage_options=storage_options, engine=engine)
  File "/Users/louisebonhomme/opt/anaconda3/lib/python3.9/site-packages/pandas/io/excel/_base.py", line 1376,

## Régression OLS

### Process sur les données 
- ajout trimestres 
- split appartement / maison : appartement (prix/m2), maison (prix)
- attention : normalisation ztransform 
- log-log 

**Ajout trimestres**

In [218]:
annee_trimestre(df)

**IPL Q-1**

In [219]:
ipl = pd.read_csv('/Users/louisebonhomme/Documents/GitHub/BDC/Data/IPL.csv', sep = ';').iloc[3:,:2]
ipl.columns = ['date','IPL']
ipl["IPL"].astype('float64')
ipl['quarter'] = ipl['date'].apply(lambda x : x[:4]+'_Q'+x[-1])
ipl['IPL_{Q-1}'] = ipl['IPL'].shift(-1)
ipl.drop(['date'], axis = 1, inplace = True)

In [220]:
ipl

,IPL,quarter,IPL_{Q-1}
3,134.3,2022_Q3,130.8
4,130.8,2022_Q2,128.6
5,128.6,2022_Q1,127.5
6,127.5,2021_Q4,126.2
7,126.2,2021_Q3,122.5
8,122.5,2021_Q2,120.2
9,120.2,2021_Q1,119.3
10,119.3,2020_Q4,117.8
11,117.8,2020_Q3,115.5
12,115.5,2020_Q2,113.9


In [221]:
df = df.merge(ipl, on = 'quarter')

In [225]:
df[['IPL_{Q-1}']].isnull().sum(axis=0)

IPL_{Q-1}    0
dtype: int64

**Split appart/maison**

In [240]:
df_maison , df_appart = split_appart_maison(df)

Nombre de maisons : 493
Nombre d'apparts : 144048


**Normalisation des données**

In [241]:
zTransform(df_appart, liste_vars = ['surface_terrain', 'surface_reelle_bati', 'nombre_pieces_principales', 'IPL_{Q-1}'], prix = 'Prix_m2')

[2.06104909e-01 5.61854174e+01 2.53404421e+00 1.13910169e+02
 1.03531091e+04]


/Users/louisebonhomme/Documents/GitHub/BDC/Helpers/preprocessing.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[liste_vars] = scaler.transform(data[liste_vars])


**Echelle log-log**

In [238]:
#log_var(df_appart, liste_vars = ['surface_terrain', 'surface_reelle_bati', 'nombre_pieces_principales', 'IPL_{Q-1}','Prix_m2'])

/Users/louisebonhomme/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arraylike.py:397: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/louisebonhomme/Documents/GitHub/BDC/Helpers/preprocessing.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[new_variable] = np.log(data[variable])
/Users/louisebonhomme/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arraylike.py:397: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/louisebonhomme/Documents/GitHub/BDC/Helpers/preprocessing.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer

In [239]:
df_appart

,id_mutation,date_mutation,code_type_local,code_commune,surface_terrain,surface_reelle_bati,nombre_pieces_principales,nature_culture,valeur_fonciere,latitude,...,prix_m2_moy_terrain,num_trimestre,quarter,IPL,IPL_{Q-1},surface_terrain_log,surface_reelle_bati_log,nombre_pieces_principales_log,IPL_{Q-1}_log,Prix_m2_log
0,2017-731653,2017-07-04,2.0,75103,-0.02239,-0.333378,0.318821,0,571110.0,48.858043,...,0.0,3,2017_Q3,105.7,-1.323180,NaN,NaN,-1.143126,NaN,0.161223
1,2017-731668,2017-07-07,2.0,75103,-0.02239,0.747689,0.318821,0,1414600.0,48.857626,...,0.0,3,2017_Q3,105.7,-1.323180,NaN,-0.290768,-1.143126,NaN,0.728484
2,2017-731783,2017-07-19,2.0,75103,-0.02239,-0.920915,-1.049638,0,215000.0,48.856136,...,0.0,3,2017_Q3,105.7,-1.323180,NaN,NaN,NaN,NaN,-0.185540
3,2017-731808,2017-07-27,2.0,75103,-0.02239,0.371666,0.318821,0,598000.0,48.857626,...,0.0,3,2017_Q3,105.7,-1.323180,NaN,-0.989760,-1.143126,NaN,NaN
4,2017-731873,2017-07-28,2.0,75103,-0.02239,-0.568393,-0.365409,0,360000.0,48.856914,...,0.0,3,2017_Q3,105.7,-1.323180,NaN,NaN,NaN,NaN,-1.124636
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144536,2022-535864,2022-06-09,2.0,75113,-0.02239,-0.145366,-0.365409,0,530000.0,48.834122,...,0.0,2,2022_Q2,130.8,1.867145,NaN,NaN,NaN,0.624411,-2.414624
144537,2022-536755,2022-06-17,2.0,75113,-0.02239,-0.497888,-0.365409,0,375000.0,48.836186,...,0.0,2,2022_Q2,130.8,1.867145,NaN,NaN,NaN,0.624411,-2.034203
144538,2022-532413,2022-05-04,2.0,75112,-0.02239,-0.709402,-1.049638,0,269450.0,48.840657,...,0.0,2,2022_Q2,130.8,1.867145,NaN,NaN,NaN,0.624411,-5.586345
144539,2022-533612,2022-05-18,2.0,75112,-0.02239,0.536176,1.687280,0,860000.0,48.841395,...,0.0,2,2022_Q2,130.8,1.867145,NaN,-0.623293,0.523118,0.624411,-1.645111


### Split données 

There are 20 quarters -> using 15 quarters as train and 5 as test

In [242]:
missing_df = df_appart.isnull().sum(axis=0).reset_index()
missing_df.columns = ['variable', 'missing values']
missing_df['filling factor (%)']=(df_appart.shape[0]-missing_df['missing values'])/df_appart.shape[0]*100
missing_df['type'] = df_appart.dtypes.to_numpy()
missing_df

,variable,missing values,filling factor (%),type
0,id_mutation,0,100.0,object
1,date_mutation,0,100.0,datetime64[ns]
2,code_type_local,0,100.0,float64
3,code_commune,0,100.0,int64
4,surface_terrain,0,100.0,float64
5,surface_reelle_bati,0,100.0,float64
6,nombre_pieces_principales,0,100.0,float64
7,nature_culture,0,100.0,object
8,valeur_fonciere,0,100.0,float64
9,latitude,0,100.0,float64


In [229]:
df_appart.sort_values(by = 'quarter')

,id_mutation,date_mutation,code_type_local,code_commune,surface_terrain,surface_reelle_bati,nombre_pieces_principales,nature_culture,valeur_fonciere,latitude,...,prix_m2_moy_terrain,num_trimestre,quarter,IPL,IPL_{Q-1},surface_terrain_log,surface_reelle_bati_log,nombre_pieces_principales_log,IPL_{Q-1}_log,Prix_m2_log
0,2017-731653,2017-07-04,2.0,75103,-0.02239,-0.333378,0.318821,0,571110.0,48.858043,...,0.0,3,2017_Q3,105.7,-1.323180,NaN,NaN,-1.143126,NaN,0.161223
5218,2017-747601,2017-09-26,2.0,75117,-0.02239,1.147214,1.687280,0,655000.0,48.878511,...,0.0,3,2017_Q3,105.7,-1.323180,NaN,0.137337,0.523118,NaN,NaN
5217,2017-747339,2017-09-22,2.0,75117,-0.02239,0.042645,-0.365409,0,633000.0,48.877921,...,0.0,3,2017_Q3,105.7,-1.323180,NaN,-3.154837,NaN,NaN,-1.594413
5216,2017-747273,2017-08-25,2.0,75117,-0.02239,1.946264,1.687280,0,1603000.0,48.878021,...,0.0,3,2017_Q3,105.7,-1.323180,NaN,0.665912,0.523118,NaN,-0.850924
5215,2017-747176,2017-08-21,2.0,75117,-0.02239,1.217719,2.371509,0,655000.0,48.878511,...,0.0,3,2017_Q3,105.7,-1.323180,NaN,0.196979,0.863527,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139098,2022-522157,2022-05-19,2.0,75104,-0.02239,-0.380381,-0.365409,0,538500.0,48.856636,...,0.0,2,2022_Q2,130.8,1.867145,NaN,NaN,NaN,0.624411,0.118612
139097,2022-521786,2022-05-17,2.0,75104,-0.02239,-0.615396,-1.049638,0,320000.0,48.857018,...,0.0,2,2022_Q2,130.8,1.867145,NaN,NaN,NaN,0.624411,-2.175587
139096,2022-520966,2022-04-27,2.0,75104,-0.02239,-0.826909,-0.365409,0,206000.0,48.856968,...,0.0,2,2022_Q2,130.8,1.867145,NaN,NaN,NaN,0.624411,NaN
139109,2022-524047,2022-06-02,2.0,75120,-0.02239,-0.732903,-0.365409,0,203000.0,48.869339,...,0.0,2,2022_Q2,130.8,1.867145,NaN,NaN,NaN,0.624411,NaN


In [243]:
df_train = df_appart[df_appart['date_mutation'] < '2021-04-01']
df_test = df_appart[df_appart['date_mutation'] >= '2021-04-01']

print(f'Train size : {round(df_train.shape[0]/df.shape[0], 2)*100}%')
print(f'Test size : {round(df_test.shape[0]/df.shape[0], 2)*100}%')

Train size : 73.0%
Test size : 26.0%


**Régression OLS**

Variables utilisées : 
- Y = prix/m2
- surface_terrain ?
- surface_reelle_bati
- nombre_pieces_principales
- ajouter Y_t-1 ? 

In [248]:
Features = ['surface_terrain', 'surface_reelle_bati', 'nombre_pieces_principales', 'IPL_{Q-1}']
Label = 'Prix_m2'

Y_train = df_train[Label]
X_train = df_train[Features]

Y_test = df_test[Label]
X_test = df_test[Features]

In [249]:
X_train = sm.add_constant(X_train)
X_test = sm.add_constant(X_test)

model = sm.OLS(Y_train, X_train).fit()

In [250]:
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                Prix_m2   R-squared:                       0.064
Model:                            OLS   Adj. R-squared:                  0.064
Method:                 Least Squares   F-statistic:                     1814.
Date:                Tue, 07 Mar 2023   Prob (F-statistic):               0.00
Time:                        11:42:46   Log-Likelihood:            -1.4656e+05
No. Observations:              106171   AIC:                         2.931e+05
Df Residuals:                  106166   BIC:                         2.932e+05
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                                coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------
const                 

In [254]:
mean_squared_error(Y_test, model.predict(X_test))

1.1070087458229645